In [1]:
import os
from huggingface_hub import login

login(token= os.getenv("hf_token"))

C:\Users\aravi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
import torch
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model = "google/gemma-3-270m"
adapter_path = "A:/gemma270-rag/finetune/gemma_finetuned"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)

model = AutoModelForCausalLM.from_pretrained(
    base_model, 
    device_map="cpu", 
    torch_dtype=torch.float32
)

model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, adapter_path)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
C:\Users\aravi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\peft\peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.wei

In [7]:
prompt = "I feel like I'm not healed enough to be in a relationship. I have too much baggage. with reference to thirukkural"
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)



print(tokenizer.decode(outputs[0], skip_special_tokens=True))

I feel like I'm not healed enough to be in a relationship. I have too much baggage. with reference to thirukkural, i know this from my own experience.

i think i have a lot of baggage that i have to sort through and deal with. i have to deal with my past and i have to deal with the past.

I have a lot of baggage.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to deal with the past.

I have to


In [2]:
import re

def parse_output(text):
    kural_id = re.search(r"<kural_id>(.*?)</kural_id>", text, re.DOTALL)
    kural_translation = re.search(r"<kural_translation>(.*?)</kural_translation>", text, re.DOTALL)
    explanation = re.search(r"<explanation>(.*?)</explanation>", text, re.DOTALL)

    return {
        "kural_id": kural_id.group(1).strip() if kural_id else "",
        "kural_translation": kural_translation.group(1).strip() if kural_translation else "",
        "explanation": explanation.group(1).strip() if explanation else "",
    }


In [8]:
import torch
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model = "google/gemma-3-270m"
adapter_path = "A:/gemma270-rag/finetune/updated_finetune"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)

model = AutoModelForCausalLM.from_pretrained(
    base_model, 
    device_map="cpu", 
    torch_dtype=torch.float32
)

model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, adapter_path)

# prompt = "I feel like I'm not healed enough to be in a relationship. I have too much baggage."
# inputs = tokenizer(prompt, return_tensors="pt")

#Inference - 1 == looks good but not actually
# generated = model.generate(
#     input_ids=inputs["input_ids"],
#     max_length=50,
#     do_sample=True,
#     top_k=50,
#     top_p=0.9,
#     temperature=0.7,
#     repetition_penalty=2.0,
#     num_return_sequences=1,
#     pad_token_id=tokenizer.eos_token_id
# )

#Inference two - actually good
# generated = model.generate(
#     input_ids=inputs["input_ids"],
#     max_length=100,
#     do_sample=False,
#     repetition_penalty=1.5,
#     num_return_sequences=1,
#     pad_token_id=tokenizer.eos_token_id
# )

prompt = (
    "Query: I'm so bad at science. I don't understand how anything works.\n"
    "Answer in the following structured format:\n"
    "<kural_id></kural_id>\n<kural_translation></kural_translation>\n<explanation></explanation>"
)

inputs = tokenizer(prompt, return_tensors="pt")

generated = model.generate(
    input_ids=inputs["input_ids"],
    max_length=256,
    do_sample=False,
    repetition_penalty=1.5,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id
)

response = tokenizer.decode(generated[0], skip_special_tokens=True)
parsed = parse_output(response)

print(parsed)


# print(tokenizer.decode(generated[0], skip_special_tokens=True))


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{'kural_id': '', 'kural_translation': '', 'explanation': ''}


In [9]:
response

'Query: I\'m so bad at science. I don\'t understand how anything works.\nAnswer in the following structured format:\n<kural_id></kural_id>\n<kural_translation></kural_translation>\n<explanation></explanation>\n\nI have a question about this, and it is not clear to me what you mean by "science" or something like that... but if someone can help with my understanding of these things please let them know! here are some examples from different sources (which might be more than one): http://www-01257/s39684fcfdffbcebbcbaebdbfeacfaeafcaaeaefcbdaeeabdfbcbfadccbeecbdfcddcddcdeed)\n\n\n<blockquote>The first thing we need for any scientific explanation - an <b><i>atom</i></b> -- which means all matter except protons & neutrons; no other particles besides electrons etc., as well as their interactions within atoms themselves</blockquote>\n\n\n\nThis seems very much correct indeed....but then again there isn’t really enough information available on earth right now either? What exactly does “an atom”

# Finetuned with addtional tokenizer

In [4]:
import torch
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model = "google/gemma-3-270m"
adapter_path = "A:/gemma270-rag/finetune/finetuned_3"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)

model = AutoModelForCausalLM.from_pretrained(
    base_model, 
    device_map="cpu", 
    torch_dtype=torch.float32
)

model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, adapter_path)

prompt = ("""Example 1:
Query: I'm struggling to learn about time management.
Answer in the following structured format:
<kural_id>402</kural_id>
<kural_translation>...</kural_translation>
<explanation>...</explanation>

Example 2:
Query: I feel like I can't handle stress at work.
Answer in the following structured format:
<kural_id>394</kural_id>
<kural_translation>...</kural_translation>
<explanation>...</explanation>

Now your turn:
Query: I'm so bad at science. I don't understand how anything works.
Answer in the following structured format:
<kural_id>""")

inputs = tokenizer(prompt, return_tensors="pt")

generated = model.generate(
    input_ids=inputs["input_ids"],
    max_length=256,
    do_sample=False,
    repetition_penalty=1.5,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id
)

response = tokenizer.decode(generated[0], skip_special_tokens=True)
parsed = parse_output(response)

print(parsed)

print("response:")
response

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{'kural_id': '', 'kural_translation': '', 'explanation': ''}
response:


"Example 1:\nQuery: I'm struggling to learn about time management.\nAnswer in the following structured format:\n402\n...\n...\n\nExample 2:\nQuery: I feel like I can't handle stress at work.\nAnswer in the following structured format:\n394\n...\n...\n\nNow your turn:\nQuery: I'm so bad at science. I don't understand how anything works.\nAnswer in the following structured format:\n568 quadrature\n quadrature... quadrature quadrature ... quadrature ... quadrature ... quadrature .... quadrature .. quadrature ..... quadrature ........... quadrature ....... quadrature ...... quadrature ........ quadrature .......... quadrature .................. quadrature .............. quadrature........... quadrature............ quadrature............. quadrature.............. quadrature............... quadrature................................. quadrature........................ quadrature(s) (a). b, c and d are all real numbers; a is not defined by this expression but it could be any number between -in